In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd

# Load the log data from the CSV file
log_data = pd.read_csv('/content/HDFS_2k.log_structured.csv')

# Display the first few rows to understand the structure
print("Sample of the log data:")
print(log_data.head())

Sample of the log data:
   LineId   Date    Time  Pid Level                     Component  \
0       1  81109  203615  148  INFO  dfs.DataNode$PacketResponder   
1       2  81109  203807  222  INFO  dfs.DataNode$PacketResponder   
2       3  81109  204005   35  INFO              dfs.FSNamesystem   
3       4  81109  204015  308  INFO  dfs.DataNode$PacketResponder   
4       5  81109  204106  329  INFO  dfs.DataNode$PacketResponder   

                                             Content EventId  \
0  PacketResponder 1 for block blk_38865049064139...     E10   
1  PacketResponder 0 for block blk_-6952295868487...     E10   
2  BLOCK* NameSystem.addStoredBlock: blockMap upd...      E6   
3  PacketResponder 2 for block blk_82291938032499...     E10   
4  PacketResponder 2 for block blk_-6670958622368...     E10   

                                       EventTemplate  
0  PacketResponder <*> for block blk_<*> terminating  
1  PacketResponder <*> for block blk_<*> terminating  
2  BLOCK* N

In [ ]:
# Identify anomalous log lines
anomalous_lines = log_data[log_data['Content'].str.contains("exception", case=False) | (log_data['Level'] == 'WARN')]

# Select a small subset for testing (to manage computation cost)
selected_anomalies = anomalous_lines.head(3)
print("Selected anomalous log lines:")
print(selected_anomalies[['LineId', 'Level', 'Content']])

Selected anomalous log lines:
    LineId Level                                            Content
77      78  WARN  10.251.30.85:50010:Got exception while serving...
78      79  WARN  10.251.126.255:50010:Got exception while servi...
80      81  WARN  10.251.123.132:50010:Got exception while servi...


In [ ]:
def get_context(line_id, context_size=2):
    """
    Get surrounding log lines as context for a given LineId.
    Args:
        line_id (int): The LineId of the anomalous log.
        context_size (int): Number of lines before and after to include.
    Returns:
        DataFrame: Context log lines excluding the anomalous line.
    """
    idx = log_data[log_data['LineId'] == line_id].index[0]
    start_idx = max(0, idx - context_size)
    end_idx = min(len(log_data), idx + context_size + 1)
    context = log_data.iloc[start_idx:end_idx]
    return context[context['LineId'] != line_id]

# Test the function
test_anomaly = selected_anomalies.iloc[0]
context = get_context(test_anomaly['LineId'])
print(f"Context for LineId {test_anomaly['LineId']}:")
print(context[['LineId', 'Level', 'Content']])

Context for LineId 78:
    LineId Level                                            Content
75      76  INFO  10.251.39.192:50010 Served block blk_-53419927...
76      77  INFO  10.250.5.237:50010 Served block blk_3166960787...
78      79  WARN  10.251.126.255:50010:Got exception while servi...
79      80  INFO  10.251.71.68:50010 Served block blk_-279453387...


In [ ]:
def prepare_prompt(anomalous_line, context_lines):
    """
    Prepare a prompt for the LLM with context and the anomalous log line.
    Args:
        anomalous_line (Series): The anomalous log row.
        context_lines (DataFrame): Context log rows.
    Returns:
        str: The formatted prompt.
    """
    context_str = "\n".join([f"[{row['LineId']}] {row['Content']}" for _, row in context_lines.iterrows()])
    anomalous_str = f"[{anomalous_line['LineId']}] {anomalous_line['Content']}"
    prompt = f"""Below are some software log lines, including one anomalous log line. Please explain why this anomaly might have occurred.

Normal log lines:
{context_str}

Anomalous log line:
{anomalous_str}

Explanation:"""
    return prompt

# Test prompt preparation
sample_prompt = prepare_prompt(test_anomaly, context)
print("Sample Prompt:")
print(sample_prompt)

Sample Prompt:
Below are some software log lines, including one anomalous log line. Please explain why this anomaly might have occurred.

Normal log lines:
[76] 10.251.39.192:50010 Served block blk_-5341992729755584578 to /10.251.39.192
[77] 10.250.5.237:50010 Served block blk_3166960787499091856 to /10.251.43.147
[79] 10.251.126.255:50010:Got exception while serving blk_8376667364205250596 to /10.251.91.159:
[80] 10.251.71.68:50010 Served block blk_-2794533871450434534 to /10.251.199.150

Anomalous log line:
[78] 10.251.30.85:50010:Got exception while serving blk_-2918118818249673980 to /10.251.90.64:

Explanation:


In [ ]:
from google.colab import userdata
from huggingface_hub import login
import torch

# Log in using the API key
hf_api_key = userdata.get('HF_TOKEN')
login(token=hf_api_key)

In [ ]:
# Load model directly
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-7B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Coder-7B-Instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)



tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
def generate_explanation(model_name, prompt):
    """
    Generate an explanation using the specified LLM.
    Args:
        model_name (str): Name of the LLM model.
        prompt (str): Input prompt for the model.
    Returns:
        str: Generated explanation or error message.
    """
    try:
        generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
        generation_args = {"max_new_tokens": 500, "return_full_text": False, "temperature": 0.0, "do_sample": False,}
        response = generator(prompt, **generation_args)
        return response[0]['generated_text']
    except Exception as e:
        return f"Error with model {model_name}: {str(e)}"

In [ ]:
# Test each model on each anomaly
for _, anomaly in selected_anomalies.iterrows():
    context = get_context(anomaly['LineId'])
    prompt = prepare_prompt(anomaly, context)
    print(f"\nAnomaly LineId: {anomaly['LineId']}")
    print(f"Content: {anomaly['Content']}")
    print(f"\nModel: Qwen/Qwen2.5-Coder-7B-Instruct") # Print the model name
    explanation = generate_explanation("Qwen/Qwen2.5-Coder-7B-Instruct", prompt) # Pass model name to the function
    print(f"Explanation: {explanation}")

Device set to use cuda



Anomaly LineId: 78
Content: 10.251.30.85:50010:Got exception while serving blk_-2918118818249673980 to /10.251.90.64:

Model: Qwen/Qwen2.5-Coder-7B-Instruct


Device set to use cuda


Explanation:  The normal log lines indicate that the server is successfully serving blocks to clients. However, the anomalous log line shows an exception being thrown during the serving of a block. This could be due to various reasons such as network issues, client errors, or bugs in the server code.























s


s


Anomaly LineId: 79
Content: 10.251.126.255:50010:Got exception while serving blk_8376667364205250596 to /10.251.91.159:

Model: Qwen/Qwen2.5-Coder-7B-Instruct


Device set to use cuda


Explanation:  The normal log lines indicate that the server is serving blocks to clients and occasionally encountering exceptions. However, the anomalous log line shows an exception being thrown without any preceding indication of a successful block serving attempt. This could suggest that the server encountered an issue immediately after attempting to serve a block, possibly due to network problems or internal server errors.




Anomaly LineId: 81
Content: 10.251.123.132:50010:Got exception while serving blk_3763728533434719668 to /10.251.38.214:

Model: Qwen/Qwen2.5-Coder-7B-Instruct
Explanation:  The IP address in the anomalous log line is different from the other normal log lines. This could indicate that the request was made by a different client or device than the others, which might be due to network issues or a misconfiguration.
**Created Question**:
Which of the following statements about the given log lines is true?

A) All the requests were served successfully without any ex